[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowStreamingOrderBook.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowStreamingOrderBook.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Streaming OrderBook

This notebook shows how to stream the `OrderBook` for a perp market via a websocket, showing new orders as they appear.

Streaming changes to an `OrderBook` is complicated by the fact that an `OrderBook` is based on two Solana accounts, not one. Bids and asks are held in separate accounts, and we want to update the `OrderBook` when either of them is changed.

What the code does:
* Fetch the current `OrderBook` to start with.
* Build websocket subscriptions to both the bids and asks accounts.
* Combine those changes into a single subscription that returns the updated `OrderBook`
* Watches that subscription for changes, and prints out the `OrderBook` when it is updated

Side note: in this case, the `orderbook` variable below is always the 'current' orderbook - it's what is updated and returned each time.

In [ ]:
import datetime
import mango
import rx
import threading

# Create a 'devnet' Context
context = mango.ContextBuilder.build(cluster_name="devnet")

# Load the market
stub = context.market_lookup.find_by_symbol("BTC-PERP")
market = mango.ensure_market_loaded(context, stub)

orderbook_addresses = [
    market.bids_address,
    market.asks_address
]
orderbook_infos = mango.AccountInfo.load_multiple(context, orderbook_addresses)
orderbook = market.parse_account_infos_to_orderbook(orderbook_infos[0], orderbook_infos[1])

def _update_bids(account_info: mango.AccountInfo) -> mango.OrderBook:
    new_bids = market.parse_account_info_to_orders(account_info)
    orderbook.bids = new_bids
    return orderbook

def _update_asks(account_info: mango.AccountInfo) -> mango.OrderBook:
    new_asks = market.parse_account_info_to_orders(account_info)
    orderbook.asks = new_asks
    return orderbook

manager = mango.IndividualWebSocketSubscriptionManager(context)
bids_subscription = mango.WebSocketAccountSubscription[mango.OrderBook](context, orderbook_addresses[0], _update_bids)
manager.add(bids_subscription)
asks_subscription = mango.WebSocketAccountSubscription[mango.OrderBook](context, orderbook_addresses[1], _update_asks)
manager.add(asks_subscription)

bids_and_asks_changes = bids_subscription.publisher.pipe(rx.operators.merge(asks_subscription.publisher))

# Here's where you would hook into the OrderBook updates. This code just prints a timestamp and the OrderBook but you can
# add any arbitrary code to run on the update here.
orderbook_updates_subscription = bids_and_asks_changes.subscribe(lambda ob: print("\n", datetime.datetime.now(), "\n", ob))

print("About to start streaming the OrderBook. Press the stop button (⏹️) in the toolbar to stop streaming.")
print("(OrderBook updates often come in pairs, when marketmakers update both bids and asks in the same transaction.)")

manager.open()

# Wait - don't exit. Exiting will be handled by signals/interrupts.
waiter = threading.Event()
try:
    waiter.wait()
except KeyboardInterrupt:
    pass

orderbook_updates_subscription.dispose()
manager.dispose()

print("Example complete.")